## Cotton origin probabilities ##

### If i buy yarn in italy, where is more probable it comes from? ###

From the UN Comtrade Database, searching with HS code for cotton yarn (HS 5205 = cotton yarn, not sewing thread, containing ≥85% cotton), we retrieved italian imports for this good.



In [29]:
import pandas as pd

df_italy_cotton_inports = pd.read_csv("2024_italy_trade_data.csv", quotechar='"', na_values=["", " "])

print(df_italy_cotton_inports.head())

  typeCode freqCode  refPeriodId  refYear  refMonth  period  reporterCode  \
0        C        A     20240101     2024        52    2024             8   
1        C        A     20240101     2024        52    2024            31   
2        C        A     20240101     2024        52    2024            32   
3        C        A     20240101     2024        52    2024            36   
4        C        A     20240101     2024        52    2024            40   

  reporterISO reporterDesc flowCode  ...       netWgt  isNetWgtEstimated  \
0         ALB      Albania        M  ...  59003.53.00              False   
1         AZE   Azerbaijan        M  ...    643.36.00              False   
2         ARG    Argentina        M  ...          3.6              False   
3         AUS    Australia        M  ...   9566.36.00              False   
4         AUT      Austria        M  ...       556944              False   

  grossWgt isGrossWgtEstimated       cifvalue      fobvalue   primaryValue  \
0 

## Analysis of Italy's Cotton Imports

Let's analyze the data to understand:
1. The total value of cotton imports to Italy
2. The country with the highest import value
3. The corresponding weight for the top importing country

In [30]:
# 1. Calculate the total value of cotton imports to Italy (sum of CIF value)
# Convert cifvalue to numeric to ensure proper calculations
df_italy_cotton_inports['cifvalue'] = pd.to_numeric(df_italy_cotton_inports['cifvalue'], errors='coerce')

# Calculate the total CIF value
total_cif_value = df_italy_cotton_inports['cifvalue'].sum()

print(f"Total value of cotton imports to Italy: ${total_cif_value:,.2f} USD")

Total value of cotton imports to Italy: $50,058,609.50 USD


In [26]:
# 2. Find the country with the maximum import value
# Let's examine the isAggregate column first to understand the data structure
print("isAggregate value counts:")
print(df_italy_cotton_inports['isAggregate'].value_counts())

# It seems there might be an issue with the filtering
# Let's try a different approach and look at the reporterDesc directly
# Find the row with the maximum CIF value
max_row = df_italy_cotton_inports.loc[df_italy_cotton_inports['cifvalue'].idxmax()]

print(f"\nCountry with highest cotton import value: {max_row['reporterDesc']}")
print(f"Maximum import value: ${max_row['cifvalue']:,.2f} USD")

isAggregate value counts:
isAggregate
True    62
Name: count, dtype: int64

Country with highest cotton import value: Hungary
Maximum import value: $16,588,946.00 USD


In [28]:
# 3. Check if the country with the max import value has netWgt data
# Let's first check the netWgt data type for the max row
print(f"netWgt for {max_row['reporterDesc']}: {max_row['netWgt']}")
print(f"netWgt data type: {type(max_row['netWgt'])}")

# Try to convert this specific value to numeric
try:
    netWgt_value = pd.to_numeric(max_row['netWgt'])
    print(f"Net weight of cotton imported from {max_row['reporterDesc']}: {netWgt_value:,.2f} kg")
    # Calculate the price per kg if applicable
    if netWgt_value > 0:
        price_per_kg = max_row['cifvalue'] / netWgt_value
        print(f"Average price: ${price_per_kg:.2f} USD per kg")
    else:
        print("Net weight is zero or negative, can't calculate price per kg.")
except (ValueError, TypeError):
    print(f"Could not convert netWgt to numeric value for {max_row['reporterDesc']}")

netWgt for Hungary: 0
netWgt data type: <class 'str'>
Net weight of cotton imported from Hungary: 0.00 kg
Net weight is zero or negative, can't calculate price per kg.


## Summary of Cotton Imports to Italy

From our analysis, we can conclude:

1. **Total Import Value**: The total CIF value of cotton imports to Italy is approximately $50 million USD.

2. **Top Importing Country**: Hungary is the country with the highest cotton import value to Italy, with imports valued at about $16.6 million USD.

3. **Net Weight Data**: While Hungary reports a net weight value, it appears to be zero in the dataset, suggesting either missing data or that the imports might not be physical cotton yarn but could represent financial transactions or other types of trade arrangements.

This data helps us understand the cotton import landscape for Italy, with Hungary being the dominant trading partner in this category.